In [6]:
from collections import Counter
import yaml

In [8]:
def load_yaml(yaml_path: str) -> dict:
    """通过id找到名称

    Args:
        yaml_path (str): yaml文件路径

    Returns:
        yaml (dict)
    """
    with open(yaml_path, 'r', encoding='utf-8') as f:
        y = yaml.load(f, Loader=yaml.FullLoader)

    return y

In [11]:
remap_dict: dict = load_yaml(r"../weights/yolov5m-sgd-e300-bg1000/remap.yaml")
remap_dict["remap"]

{0: 0,
 1: 23,
 2: 23,
 3: 23,
 4: 20,
 5: 20,
 6: 21,
 7: 21,
 8: 14,
 9: 14,
 10: 18,
 11: 11,
 12: 0,
 13: 0,
 14: 0,
 15: 0,
 16: 0,
 17: 23,
 18: 0,
 19: 23}

In [82]:
data = {
    'detect': [
        {'class_index': 10, 'class': 'mian_ya', 'confidence': 0.935646653175354, 'box': [0, 1011, 546, 1562]},
        {'class_index': 10, 'class': 'mian_ya', 'confidence': 0.8366711735725403, 'box': [154, 539, 442, 1117]},
        {'class_index': 10, 'class': 'mian_ya', 'confidence': 0.7807613611221313, 'box': [290, 1481, 1131, 2015]},
        {'class_index': 10, 'class': 'mian_ya', 'confidence': 0.4629181921482086, 'box': [427, 374, 983, 1322]},
        {'class_index': 14, 'class': 'piao_chong_yong', 'confidence': 0.6198558211326599, 'box': [821, 346, 838, 380]},
        {'class_index': 15, 'class': 'piao_chong_you_chong', 'confidence': 0.583463191986084, 'box': [603, 652, 618, 675]},
        {'class_index': 14, 'class': 'piao_chong_yong', 'confidence': 0.663463191986084, 'box': [500, 449, 620, 675]}
        ],
    'count': {10: 4, 14: 1, 15: 1},
    'image_size': (2016, 1134, 3)
}

In [83]:
def remap(data: dict, remap_dict: dict) -> dict:
    """将预测id转换为数据库id,删除不需要的数据

    Args:
        data (dict):       预测数据
        remap_dict (dict): origin id to database id dict

    Returns:
        dict:              remap的数据
    """
    new_count = []
    new_box   = []
    for box in data["detect"]:
        # 去除名字
        box.pop("class")
        # 映射id
        box["class_index"] = remap_dict[box["class_index"]]

        # 忽略new_id为0的类别,这个类别不要
        if box["class_index"] == 0:
            # 不能直接remove,会导致训练数据长度变化,后面一个的数据读取不到
            continue

        new_box.append(box)
        new_count.append(box["class_index"])

    # reamp id 计数
    data["detect"] = new_box
    data["count"] = dict(Counter(new_count))

    return data

In [84]:
remap(data, remap_dict["remap"])

{'detect': [{'class_index': 18,
   'confidence': 0.935646653175354,
   'box': [0, 1011, 546, 1562]},
  {'class_index': 18,
   'confidence': 0.8366711735725403,
   'box': [154, 539, 442, 1117]},
  {'class_index': 18,
   'confidence': 0.7807613611221313,
   'box': [290, 1481, 1131, 2015]},
  {'class_index': 18,
   'confidence': 0.4629181921482086,
   'box': [427, 374, 983, 1322]}],
 'count': {18: 4},
 'image_size': (2016, 1134, 3)}

# 测试python动态删除数据

In [10]:
test_data = [
    {"name": "A"},
    {"name": "B"},
    {"name": "C"},
    {"name": "D"},
    {"name": "E"},
]

new_test_data = []
for i, d in enumerate(test_data):
    print(i, d["name"])
    if d["name"] == "C":
        test_data.remove(d)
        continue
    new_test_data.append(d)
    print(len(test_data))
# name == c 时
# 会pop掉自己,长度变短,但是索引累加,会导致跳过数据
# ABCDE index=2 => C
# index += 1
# ABDE  index=3 => E 这一次就是E了,直接跳过了D,动态删除C会导致下一个D被忽略
new_test_data # 可以看到虽然只删除了C,D也被跳过了

0 A
5
1 B
5
2 C
3 E
4


[{'name': 'A'}, {'name': 'B'}, {'name': 'E'}]

# 解决办法1: 倒序删除,删除一个数据,其他数据向前移动,不影响前面的数据

In [ ]:
test_data = [
    {"name": "A"},
    {"name": "B"},
    {"name": "C"},
    {"name": "D"},
    {"name": "E"},
]

new_test_data = []
for i, d in enumerate(test_data[::-1]):
    print(i, d["name"])
    if d["name"] == "C":
        test_data.remove(d)
        continue
    new_test_data.append(d)
    print(len(test_data))
new_test_data[::-1]

0 E
5
1 D
5
2 C
3 B
4
4 A
4


[{'name': 'A'}, {'name': 'B'}, {'name': 'D'}, {'name': 'E'}]

# 解决办法2: 使用新数组存放新数据,不删除旧数据

In [12]:
test_data = [
    {"name": "A"},
    {"name": "B"},
    {"name": "C"},
    {"name": "D"},
    {"name": "E"},
]
new_test_data = []
for i, d in enumerate(test_data):
    print(i, d["name"])
    if d["name"] == "C":
        continue
    new_test_data.append(d)
new_test_data

0 A
1 B
2 C
3 D
4 E


[{'name': 'A'}, {'name': 'B'}, {'name': 'D'}, {'name': 'E'}]